![Logo-FIUBA](https://cms.fi.uba.ar/uploads/logo_0e621ee7ba.png)

# Trabajo práctico grupal N°1

### Ecuaciones no lineales y Sitemas de ecuaciones lineales

**Integrantes**:
- Pascual Osorio, Juan Manuel
- Caliz Blanco, Alejo Martin Ezequiel
- Scazzola, Lara
- Della Torre, Bautista
- Rodriguez, Sofia
- Rivero, Julian Tomás

In [36]:
import numpy as np
import math as mt
import pandas as pd

# Introduccion
## Trabajo Practico N°1

### Ecuaciones no lineales

En el siguiente problema nos proponen encontrar de forma numérica, la temperatura de salida de un aceite que pasa por un tanque de mezclado.

<img src="TPI2C2021Intercambiador.png" width=400 height=400 />

El cual, cumple con la siguientes relaciones:


$ΔT_{ml} = \frac{(T_h^e-T_c^s)-(T_h^s-T_c^e)}{ln(\frac{(T_h^e-T_c^s)}{(T_h^s-T_c^e)})}$<br>
Siempre debe cumplirse que $Q_c = Q_h = Q$.

<br>Esto se puede expresar como: $\frac{Q}{Q_h}-1 = 0$<br>


Reordenando las ecuaciones se puede dejar todo expresando en función de $T_h^s$ (incógnita):

$f(T_h^s) = \frac{\frac{UA(T_h^e-\frac{m_hCp_h}{m_cCp_c}(T_h^e-T_h^s)-T_c^e)-(T_h^s - T_c^e)}{ln(\frac{(T_h^e-\frac{m_hCp_h}{m_cCp_c}(T_h^e-T_h^s)-T_c^e)}{(T_h^s - T_c^e)})}}{m_hCp_h(T_h^e-T_h^s)} - 1 = 0$

## Utilizando lo aprendido sobre ecuaciones no lineales, intentamos resolver el problema

# Datos del problema:

- `U` kcal/(h m^2 °C) Coeficiente gobal de transferencia de calor
- `A` m^2 Area total de transferencia de calor

#### Datos del agua:

- `M_c`  Caudal másico de agua [kg/h]
- `Cp_c` Calor específico del agua [kcal/(h°C)]
- `Te_c` Temperatura de entrada del agua [°C]


#### Datos del aceite:

- `M_h`  Caudal másico de aceite [kg/h]
- `Cp_h` Calor específico del aceite [cal/(h°C)]
- `Te_h` Temperatura de entrada del aceite [°C]

In [37]:
#Datos del problema 

U = 585.84 #kcal/(h m^2 °C) Coeficiente gobal de transferencia de calor
A = 81.66 #m^2 Area total de transferencia de calor

#Datos del agua
M_c = 21000 #kg/h caudal másico de agua
Cp_c = 1 #kcal/(h°C)  Calor específico del agua
Te_c = 26.5 #°C Temperatura de entrada del agua

#Datos del aceite
M_h = 33070 #kg/h caudal másico de aceite
Cp_h = 0.5 #kcal/(h°C) Calor específico del aceite
Te_h = 121 #°C Temperatura de entrada del aceite

## Armado de la función

La función del problema nos resultaba bastante problemática a la hora de definirla, ya que tenía muchos términos y era muy 
larga. La solución que encontramos fue separar la función en 3 partes:

![EquationOfState](abc.png)

- **"a"**, que corresponde al numerador del numerador de toda la función
- **"b"**, que corresponde al divisor del numerador de toda la función
- **"c"**, que corresponde al divisor de toda la función

In [43]:
# ((a / b) / c) - 1

#------------------------------------------------------------------------------------------------------------------------------#

#Las variables definidas como "_old" son las que se utilizaron para definir f en la primer entrega del ejercicio
#Estas, debido a un error en el uso de paréntesis, alteraban a la función original y devolvían un resultado erróneo

#------------------------------------------------------------------------------------------------------------------------------#

#def Ts_c(t):
#    return ((M_h * Cp_h) / (M_c * Cp_c)) * (Te_h - t) + Te_c 

Q = (M_h * Cp_h) / (M_c * Cp_c)

def f(t):
    return ((a(t)/b(t)) / c(t)) - 1

def a(t):
    return U * A * (Te_h - Q * (Te_h - t) - Te_c) - (t - Te_c)

def b(t):
    return np.log(((Te_h - Q * (Te_h - t) - Te_c)) / (t - Te_c))

def c(t):
    return (M_h * Cp_h * (Te_h - t))

#------------------------------------------------------------------------------------------------------------------------------#

def a_old(t):
    return U * A * (Te_h - (Q * ((Te_h - t) - Te_c) - (t - Te_c)))

def b_old(t):
    return np.log((((Te_h) - (Q * ((Te_h - t) - Te_c))) / (t - Te_c)))

def c_old(t):
    return (M_h * Cp_h * (Te_h - t))

# Código Bisección:

In [39]:
def biseccion(a, b , tol, itmax = 100): 
        df = pd.DataFrame(columns=('ai','bi','pi','cond'))
        a = a 
        b = b
        contador = 0  
        x0, x1 = a, b 
        if(f(a) * f(b)>0):
            print("El método no resuelve este tipo de raíz")
            return
        while (contador < itmax) and (abs((x1 - x0) / x1) > tol):
            x0 = x1                                                 
            x1 = (a + b)/2
            aux = a
            aux1 = b
            if f(x1) * f(a) < 0:      
                df.loc[contador] = [a,b,(a+b)/2,'<0']
                b = x1
            else:
                df.loc[contador] = [a,b,(a+b)/2,'>0']
                a = x1
            contador += 1                      
        error = abs((x1 - x0) / x1)
        print('El método de bisección convergió en',contador,'iteraciones, y el T encontrado es:',x1)
        print("Los ultimos intervalos encontrados son: ", aux, "y ", aux1)
        return df

# Código Secante

In [40]:
def secante(a , b, tol, itmax = 100): 
        np.seterr('raise')
        df2 = pd.DataFrame(columns=('xcero','xuno','xdos'))
        x0 = a
        x1 = b
        contador = 0  
        x2 = x1-f(x1)*(x1-x0)/(f(x1)-f(x0))
        while (contador < itmax) and (abs((x1 - x0) / x1) > tol):
            df2.loc[contador] = [x0,x1,x2]
            x0 = x1                                                 
            x1 = x2
            x2 = x1-(f(x1)*(x1-x0))/(f(x1)-f(x0))
            contador += 1                      
        print('El método Secante convergió en',contador,'iteraciones. \nLa temperaturade salida es: ',x2, '°C\nLa funcion da: ', f(x2))
        return df2

## Condiciones de problema

Para que el problema tuviera sentido desde el punto de vista de las leyes de la termodinámica, sabíamos que la temperatura de entrada del agua (Fluido de servicio/refrigerante) debía ser menor que la de salida (Dado que intercambió calor con el aceite). La misma lógica es aplicable para el aceite. Dado que este actúa como fluido de proceso/refrigerado, la temperatura de salida de este debe ser menor a la de entrada. Esto nos ayudo a la hora de definir las cotas superior e inferior del intervalo de análisis. Por otro lado, también sabíamos que era termodinámicamente imposible que el aceite salga a una temperatura menor que la temperatura de entrada del agua, y en el caso del agua, que el agua saliera a una temperatura mayor que la de entrada del aceite. Esto último nos ayudo a analizar el resultado obtenido.

# Parte Bisección

### Aqui aproximamos el intervalo de la raiz de nuestra función:

In [41]:
%%time
#Siempre la Ts_h < Te_h, por lo que la cota superior del intervalo (b) es 120°C 
#Intervalo en el cual nos encontraba una raiz la biseccion.
#Mas de 120 tiraba error.
#Menos de 27 iba a una raiz que no tenia sentido en nuestro sistema.
temperaturaMinima = 27 
temperaturaMaxima = 120
tolerancia = 10**-3
maximoIteraciones = 4 #Lo corremos 4 veces para hallar un intervalo de la raiz.
biseccion(temperaturaMinima, temperaturaMaxima, tolerancia, maximoIteraciones)

El método de bisección convergió en 4 iteraciones, y el T encontrado es: 32.8125
Los ultimos intervalos encontrados son:  27 y  38.625
Wall time: 18 ms


ai      bi       pi cond
0  27     120  73.5000   <0
1  27    73.5  50.2500   <0
2  27   50.25  38.6250   <0
3  27  38.625  32.8125   >0

Ahora que tenemos un intervalo definido por el metodo de la bisección, podemos usar el metodo de la secante para refinar el resultado.

# Parte Secante

### Ahora refinamos el valor de la raiz, tomando los intervalos que nos dió la bisección, y utilizando el metodo de la secante:

In [42]:
%%time
temperaturaMinima = 27
temperaturaMaxima = 38.625
tolerancia = 10**-13
maximoIteraciones = 150
secante(temperaturaMinima, temperaturaMaxima, tolerancia, maximoIteraciones)

El método Secante convergió en 6 iteraciones. 
La temperaturade salida es:  37.15902321008196 °C
La funcion da:  -3.3306690738754696e-16
Wall time: 14 ms


xcero       xuno       xdos
0  27.000000  38.625000  36.700947
1  38.625000  36.700947  37.136786
2  36.700947  37.136786  37.159366
3  37.136786  37.159366  37.159023
4  37.159366  37.159023  37.159023
5  37.159023  37.159023  37.159023